# Метод возвращения к среднему

В этом примере рассматривается метод возвращения цены к среднему значению за счет колебаний высокочастотных компонент.

In [ ]:
from qnt.data    import load_data, write_output
from qnt.stepper import test_strategy
from qnt.stats   import calc_stat
from qnt.graph   import make_plot, make_plot_filled

In [ ]:
assets_names = ["NASDAQ:FB"]

data = load_data(min_date = "2010-01-01",
                 max_date = "2016-12-31",
                 dims     = ("time", "field", "asset"),
                 assets   = assets_names)

## Реализация

Рассмотрим график доходностей акции Facebook.

In [ ]:
prices = data.loc[:, "open", "NASDAQ:FB"].to_series().iloc[::-1].dropna()
make_plot(prices.index, prices.pct_change(1), name="daily returns", color="#17BECF")

По графику видно, что происходят колебания около среднего значения: если доходность отклонилась в одну сторону, то она возвращается обратно. Напишем стратегию, которая покупает, когда доходность (относительное изменение цены) сильно отклоняется от среднего в отрицательную область и продает, когда в положительную.

In [ ]:
def step(data):

    is_liquid = data.loc[::-1, "is_liquid", :]

    prices         = data.loc[::-1, "open", :]
    prices_shifted = prices.shift({"time": 1})
    returns        = (prices - prices_shifted)/prices_shifted

    deviation = returns.rolling({"time": 250}).std()

    large_dev = (abs(returns) > deviation).astype(int)

    weights = -returns/abs(returns) * large_dev * is_liquid.astype(int)

    weights_sum = abs(weights).sum("asset")

    weights_norm = weights / weights_sum
    weights = weights.fillna(0.0)

    return weights[-1]

init_data_length = 250

output = test_strategy(data, step=step, init_data_length=init_data_length)

## Результаты

In [ ]:
stat = calc_stat(data, output, slippage_factor=0.05)
display(stat.to_pandas().tail())

Данная идея содержит несколько проблемных моментов. Например, частая покупка-продажа акций приводит к большим потерям относительно прироста капитала, поэтому необходимо работать с более медленно обращающимися к среднему характеристиками; в случае сильного тренда цена после отклонения разворачивается к среднему значению, но и среднее значение следует тренду: открытая, согласно сигналу о продаже, короткая позиция в случае резкого роста цены приведет к потере капитала. Указанные выше замечания делают написание стратегий возвращения к среднему более сложными нежели trend-following.


## Построение графиков

In [ ]:
performance = stat.to_pandas()["equity"]
make_plot_filled(performance.index, performance, name="PnL (Equity)", type="log")

In [ ]:
UWchart = stat.to_pandas()["underwater"]
make_plot_filled(UWchart.index, UWchart, color="darkred", name="Underwater Chart", range_max= 0)

In [ ]:
SRchart = stat.to_pandas()["sharpe_ratio"].iloc[20:]
make_plot_filled(SRchart.index, SRchart, color="#F442C5", name="Rolling SR")

In [ ]:
biaschart = stat.to_pandas()["bias"]
make_plot_filled(biaschart.index, biaschart, color="#5A6351", name="Bias Chart")

## Заключение

In [ ]:
data   = load_data(min_date="2015-01-01", dims=("time", "field", "asset"), assets=assets_names)
output = test_strategy(data, step=step, init_data_length=init_data_length)
write_output(output)